In [1]:
cd ..

/Users/minghuang/Documents/git/SEA_traffic_accident_prediction


In [2]:
from data_processing_scripts import socrata_api_wrapper as saw
from data_processing_scripts import psql_helper as ph 
from data_processing_scripts import footballdb_scraper as fs
from data_processing_scripts import baseball_almanac_scraper as bas
from data_processing_scripts import wunderground_scraper as ws
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import psycopg2
import seaborn as sns
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso
from sklearn.cross_validation import cross_val_score, train_test_split
%matplotlib inline

In [33]:
query1 = "select * from glm_data where event_date between '2013-01-01' and '2014-12-31'"
query2 = "select * from glm_data where event_date > '2014-12-31'"

In [34]:
conn = psycopg2.connect(dbname='traffic_accident', user='minghuang', host='localhost')
train_df = pd.read_sql_query(query1, con=conn)
test_df = pd.read_sql_query(query2, con=conn)
conn.close()

In [35]:
train_df.head().T

,0,1,2,3,4
s_hood,Belltown,Rainier Beach,Dunlap,Mount Baker,Industrial District
event_date,2013-01-01,2013-01-01,2013-01-01,2013-01-01,2013-01-01
hour,1,1,1,1,1
dow,3,3,3,3,3
week,1,1,1,1,1
is_intersection,0,0,0,1,0
dewpoint,28,28,28,28,28
have_gustspeed,0,0,0,0,0
have_heatindex,0,0,0,0,0
humidity,85,85,85,85,85


In [60]:
#train_vals = train_df[train_df.s_hood == 'Pioneer Square']
train_vals = train_df.copy()
train_vals = train_vals.drop(['s_hood', 'event_date'], axis=1)
train_y = train_vals.pop('number_accidents').values
train_X = train_vals.values

In [61]:
train_vals.head().T

,0,1,2,3,4
hour,1.00,1.00,1.00,1.00,1.00
dow,3.00,3.00,3.00,3.00,3.00
week,1.00,1.00,1.00,1.00,1.00
is_intersection,0.00,0.00,0.00,1.00,0.00
dewpoint,28.00,28.00,28.00,28.00,28.00
have_gustspeed,0.00,0.00,0.00,0.00,0.00
have_heatindex,0.00,0.00,0.00,0.00,0.00
humidity,85.00,85.00,85.00,85.00,85.00
have_precip,0.00,0.00,0.00,0.00,0.00
pressure,30.47,30.47,30.47,30.47,30.47


In [62]:
print 'mean:', train_y.mean()
print 'var:', train_y.var()

mean: 1.11197112766
var: 0.170888106666


In [63]:
encoder = OneHotEncoder(categorical_features=[0, 1, 2])
encoded_X = encoder.fit_transform(train_X)

In [64]:
scaler = StandardScaler()
scaled_X = scaler.fit_transform(encoded_X.toarray())

In [65]:
X_const = sm.add_constant(scaled_X, prepend=False)
poisson_model = sm.GLM(train_y, X_const, family=sm.families.Poisson())
res_pois = poisson_model.fit()

In [66]:
print(res_pois.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                44056
Model:                            GLM   Df Residuals:                    43965
Model Family:                 Poisson   Df Model:                           90
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -47686.
Date:                Wed, 30 Sep 2015   Deviance:                       4501.0
Time:                        10:55:14   Pearson chi2:                 6.38e+03
No. Iterations:                   100                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1         -8.532e+10   3.04e+11     -0.280      0.779   -6.82e+11    5.11e+11
x2         -8.035e+10   2.87e+11     -0.280      0.7